### Import modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Practice pandas DataFrame indexing

One way to creata a pandas DataFrame from scratch is to pass it a dictionary as input. Note the recycling of values in columns A and B. Also remember that the columns of the dataframe can be of different types, but the entire column must be of the same type. See in column D how all of the numbers are converted to floats with 2 decimal precision.

There are several different ways of indexing rows and columns in pandas. The first is integer indexing with the `.iloc[]` method, which extracts rows and/or columns based on their numeric index.

The second is name-based indexing with the `.loc[]` method, which extracts rows and/or columns based on their name.

The next is a shortcut for name-based indexing of columns by simply enclosing the name of that column in square brackets. Note also that if you extract a single column from a DataFrame, it is returned as a Series--a one-dimensional pandas object.

In contrast, when you extract multiple columns, the subset is returned as another pandas DataFrame.

Finally, we have Boolean indexing, where you can extract all rows or columns, again using the `.loc[]` method, where the index is equal to `True`. This is often useful for subsetting rows or columns where the value in that row or column meets some specified condition (e.g., is greater than some number). 

### Load the GTEx data

Now we can start working with some real data. We obtained human multi-tissue gene expression data from the GTEx Project, by downloading it from the GTEx Data Portal. The file is gzip compressed (ends in the `.gz` extension), so we use the option `compression = "gzip"`. We also skip the first two rows, which have unique formatting and specify that the file is tab-separated.

Download the file here: https://storage.googleapis.com/gtex_analysis_pilot_v3/rna_seq_data/GTEx_Analysis_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm__Pilot_V3_patch1.gct.gz

We saw that pandas DataFrames possess some methods that can operate on entire rows or columns. For example, by passing the `axis = 1` option to the `.median()` method, we get the median expression of every gene across all samples.  

In order to make sense of PCA, we need to filter out very low expressed genes. This decision is arbitrary, but here we decided to choose genes with median expression greater than zero. we create a new variable `roi`, which records `True` if median expression is greater than zero and `False` if not.

We can then subset the genes (stored in rows of the original DataFrame) based on this Boolean Series.

The sample names from GTEx (visible as column names here) are not particularly informative. To get more information about the individual GTEx samples, we need to load some separate metadata. This is also available from the GTEx Portal.

Download the file here: https://storage.googleapis.com/gtex_analysis_pilot_v3/annotations/GTEx_Analysis_Annotations_Sample_DS__Pilot_V3.txt

For the purpose of this exercise, we are really only interested in the tissue from which each sample was obtained. So let's subset the metadata DataFrame to just the sample name and tissue name.

### Principal component analysis

Now we are ready to get started with PCA. This is a method that operates on a numeric matrix, so we need to first remove the character columns (gene ID and gene name) from the DataFrame.

I learned this by trial and error, but it is also necessary to transpose the DataFrame (switch the row/column orientation) so that we are treating the genes as the measured variables rather than the samples. In this case, we are interested in the similarity/difference among samples in their patterns of gene expression. Transposing is achieved using `DataFrame.transpose()` or `DataFrame.T`.

Another practical step prior to PCA is standardizing the input variables (genes, in this case) to have the same means and variances (in expression across samples). This helps give the genes more equal "weight" in PCA. Let's look at the current means and variances.

Then standardize using `StandardScaler().fit_transform()` and check the new means and variances. Notice that the new means are all very close to zero, and the new variances are all equal to 1.

We are finally ready to execute the PCA function! The syntax is a little bit unfamiliar, but here we are specifying that we would like to store the top 10 principal components in `pca_output`.

As expected, our output has rows equal to the number of input samples and columns equal to the number of principal components that we decided to store. We can put this output into a new pandas DataFrame.

And look at where our samples land on the new principal component axes. Here we plot the top two principal components.

The PCA output DataFrame does not note the names of the samples. To get the names, we have to grab them from the row names of our `pca_input` DataFrame.

Now we can merge our PCA output DataFrame with our metadata DataFrame, since they share a column of unique sample IDs. We use the pandas `merge()` function to achieve this.

The last step is to add color to our plot by grouping our DataFrame on tissue, then iterating over the tissues and adding to a scatter plot. The default location of the legend overlaps the plot, so we provide some options to `plt.legend()` to shift it to the right and create two columns. We figured this out by reading the documentation for the corresponding functions and searching online!